In [ ]:
suppressPackageStartupMessages(if(!require(itsmr)) install.packages("itsmr", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(astsa)) install.packages("astsa", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(TSA)) install.packages("TSA", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(urca)) install.packages("urca", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(fracdiff)) install.packages("fracdiff", repos = "http://cran.us.r-project.org"))

suppressPackageStartupMessages(if(!require(tsoutliers)) install.packages("tsoutliers", repos = "http://cran.us.r-project.org"))

suppressPackageStartupMessages(if(!require(lmtest)) install.packages("lmtest", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(car)) install.packages("car", repos = "http://cran.us.r-project.org"))

# Previsão
suppressPackageStartupMessages(if(!require(forecast)) install.packages("forecast", repos = "http://cran.us.r-project.org"))
suppressPackageStartupMessages(if(!require(tsfknn)) install.packages("tsfknn", repos = "http://cran.us.r-project.org"))
#suppressPackageStartupMessages(if(!require(BootPR)) install.packages("BootPR", repos = "http://cran.us.r-project.org"))
#suppressPackageStartupMessages(if(!require(forecastML)) install.packages("forecastML", repos = "http://cran.us.r-project.org"))

In [ ]:
# Aumentar o tamanho das figuras
options(repr.plot.width=16, repr.plot.height=12)
#options(repr.plot.width=8, repr.plot.height=6)

In [ ]:
data(AirPassengers); serie <- AirPassengers
itsmr::plotc(serie); itsmr::plota(serie)

In [ ]:
lambda <- forecast::BoxCox.lambda(serie, method=c("guerrero"), lower = 0, upper = 1) # "loglik" "guerrero"
lambda

In [ ]:
air.fit <- forecast::Arima(serie, order=c(2,1,0), seasonal=list(order=c(0,1,0),period=12), lambda=lambda)
plot(forecast::forecast(air.fit))

# Raiz unitária

[Urca: Unit Root Test and Cointegration Test](https://jtr13.github.io/cc21fall2/urca-unit-root-test-and-cointegration-test.html)

In [ ]:
tseries::adf.test(serie)

In [ ]:
?urca::ur.df

In [ ]:
summary(urca::ur.df(serie, type = "trend"))

In [ ]:
tseries::pp.test(serie)

In [ ]:
summary(urca::ur.kpss(serie,type="mu"))

In [ ]:
summary(urca::ur.kpss(serie,type="tau"))

In [ ]:
summary(urca::ur.ers(serie, type="P-test", model="trend"))

In [ ]:
HW1 <- HoltWinters(serie)
HW2 <- HoltWinters(serie, alpha=0.8, beta=0.1, gamma=0.1)
plot(serie, ylab="Passageiros", xlim=c(1954,1958))
lines(HW1$fitted[,1], lty=2, col="blue")
lines(HW2$fitted[,1], lty=2, col="red")

In [ ]:
HW1.pred <- predict(HW1, 12, prediction.interval = TRUE, level=0.95)
plot(serie, ylab="Passageiros", xlim=c(1955.1,1962.1))
lines(HW1$fitted[,1], lty=2, col="blue")
lines(HW1.pred[,1], col="red") # Previsão
lines(HW1.pred[,2], lty=2, col="green") # Limite superior do intervalo
lines(HW1.pred[,3], lty=2, col="green") # Limite inferior do intervalo

In [ ]:
HW3 <- HoltWinters(serie, seasonal = "multiplicative")
HW3.pred <- predict(HW3, 12, prediction.interval = TRUE, level=0.95)
plot(serie, ylab="Passageiros", xlim=c(1955.1,1962.1))
lines(HW3$fitted[,1], lty=2, col="blue")
lines(HW3.pred[,1], col="red")
lines(HW3.pred[,2], lty=2, col="green")
lines(HW3.pred[,3], lty=2, col="green")

In [ ]:
HW1_forecast <- forecast(HW1, h=12, level=c(80,95))
plot(HW1_forecast, xlim=c(1955.1,1962.1))
lines(HW1_forecast$fitted, lty=2, col="blue")

In [ ]:
itsmr::arar(serie, h=12, opt=2)

In [ ]:
autoplot(forecast::snaive(serie, h=12, bootstrap=TRUE))

In [ ]:
pred <- tsfknn::knn_forecasting(serie, h = 12, lags = 1:12, k = 2)
pred$prediction
plot(pred)

# Tratamento de dados atípicos

In [ ]:
n<-1000
x<-arima.sim(n=n,list(order=c(1,0,0),ar=0.5))

In [ ]:
itsmr::plotc(x)

In [ ]:
fit.x<-arima(x, order=c(1, 0, 0), include.mean = FALSE)
coeftest(fit.x)

In [ ]:
itsmr::plotc(fitted(fit.x),x)

In [ ]:
y<-x
y[n*0.5]<-y[n*0.5]+10
itsmr::plotc(y)

In [ ]:
forecast::tsoutliers(y)

In [ ]:
itsmr::plotc(forecast::tsclean(y),y)

In [ ]:
itsmr::plota(x); itsmr::plota(y)

In [ ]:
fit.y<-arima(y, order=c(1, 0, 0), include.mean = FALSE)
lmtest::coeftest(fit.y)

In [ ]:
itsmr::plotc(fitted(fit.y),y)

In [ ]:
tc <- rep(0, n)
tc[n*0.5] <- 1
ao <- filter(tc, filter = 0, method = "recursive")
plot(ao, main = "Outlier Aditivo - TC delta = 0", type = "l")

In [ ]:
fit.interv<-arima(y, order=c(1, 0, 0), xreg=tc, include.mean = FALSE)
coeftest(fit.interv)

In [ ]:
itsmr::plotc(y-fit.interv$residuals,y)

In [ ]:
ao_effect <- fit.interv$coef[2]*tc

In [ ]:
fit.interv$coef[2]

In [ ]:
ao_effect_ts <- ts(ao_effect)

# Substraindo o efeito da intervenção
excess_wo_ts <- y - ao_effect_ts

plot(cbind(y, excess_wo_ts, ao_effect_ts), main="Serie original e corrigida por outliers")

In [ ]:
plotc(excess_wo_ts,x)

## Usando o pacote tsoutliers

In [ ]:
outliers_excess_ts <- tso(y, types = c("TC", "AO", "LS", "IO", "SLS"))
outliers_excess_ts

In [ ]:
plot(outliers_excess_ts)

In [ ]:
outliers_excess_ts$outliers

In [ ]:
(outliers_idx <- outliers_excess_ts$outliers$ind)

In [ ]:
mo_tc <- outliers("AO", outliers_idx); mo_tc

In [ ]:
tc <- outliers.effects(mo_tc, n)

In [ ]:
plot(tc,type='l')

In [ ]:
omega_hat <- as.numeric(outliers_excess_ts$outliers["coefhat"])

# Calculando vetor que representa o efeito da intervenção
ao_effect <- omega_hat*tc

ao_effect_ts <- ts(ao_effect)

# Substraindo o efeito da intervenção
excess_wo_ts <- y - ao_effect_ts

plot(cbind(y, excess_wo_ts, ao_effect_ts), main="Serie original e corrigida por outliers")

In [ ]:
plotc(excess_wo_ts,x)

### Mudança transitória

\begin{equation}
\begin{aligned}
f_t(T) = \dfrac{\omega B}{1 – \delta B} \mathbb{I}_{t}(T)
\end{aligned}
\end{equation}

In [ ]:
tc_0_4 <- filter(tc, filter = 0.4, method = "recursive")
tc_0_8 <- filter(tc, filter = 0.8, method = "recursive")
plot(tc_0_4[400:600], main = "TC delta = 0.4", type = "l") # Intervalo [400,600]
plot(tc_0_8[400:600], main = "TC delta = 0.8", type = "l")

### Mudança permanente

In [ ]:
tc <- rep(0, n)
tc[n*0.5] <- 1
ls <- filter(tc, filter = 1, method = "recursive")
plot(ls, main = "Level Shift (Mudança de nivel) - TC delta = 1", type = "l")

In [ ]:
url <- "https://bit.ly/3l3MHlm"
abhutondot <- read.csv(url, header=TRUE)

boys_ts <- ts(abhutondot$boys, frequency=1, start = abhutondot$year[1])
girls_ts <- ts(abhutondot$girls, frequency=1, start = abhutondot$year[1])

delta_ts <- boys_ts - girls_ts
excess_ts <- delta_ts/girls_ts
plot(excess_ts)

In [ ]:
outliers_excess_ts <- tso(excess_ts, types = c("TC", "AO", "LS", "IO", "SLS"))
outliers_excess_ts

In [ ]:
plot(outliers_excess_ts)

In [ ]:
outliers_excess_ts$outliers

In [ ]:
# Indice ou posição do valor atípico
(outliers_idx <- outliers_excess_ts$outliers$ind)

In [ ]:
# Tempo em que foi observado o valor atípico
outliers_excess_ts$outliers$time

In [ ]:
# Tamanho de amostra
n <- length(excess_ts)

In [ ]:
# transient change outlier at the same time index as found for our time series
mo_tc <- outliers("TC", outliers_idx)

In [ ]:
# transient change effect data is stored into a one-column matrix, tc
tc <- outliers.effects(mo_tc, n)

In [ ]:
plot(tc,type='l')

In [ ]:
# converting to a number
coefhat <- as.numeric(outliers_excess_ts$outliers["coefhat"])

# obtaining the transient change data with same magnitude as determined by the tso() function
tc_effect <- coefhat*tc

# definining a time series for the transient change data
tc_effect_ts <- ts(tc_effect, frequency = frequency(excess_ts), start = start(excess_ts))

# subtracting the transient change intervention to the original time series, obtaining the pre-intervention time series
excess_wo_ts <- excess_ts - tc_effect_ts

# plot of the original, the pre-intervention and transient change time series
plot(cbind(excess_ts, excess_wo_ts, tc_effect_ts))

In [ ]:
plotc(tc_effect_ts+excess_wo_ts)

In [ ]:
plotc(excess_ts)